In [1]:
import pandas as pd
import numpy as np
#import klib 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

In [2]:
test=pd.read_csv('test - test.csv')

In [3]:
test

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1,1043,1,1.8,1,14,0,5,0.1,193,...,16,226,1412,3476,12,7,2,0,1,0
1,2,841,1,0.5,1,4,1,61,0.8,191,...,12,746,857,3895,6,0,7,1,0,0
2,3,1807,1,2.8,0,1,0,27,0.9,186,...,4,1270,1366,2396,17,10,10,0,1,1
3,4,1546,0,0.5,1,18,1,25,0.5,96,...,20,295,1752,3893,10,0,7,1,1,0
4,5,1434,0,1.4,0,11,1,49,0.5,108,...,18,749,810,1773,15,8,7,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,1700,1,1.9,0,0,1,54,0.5,170,...,17,644,913,2121,14,8,15,1,1,0
996,997,609,0,1.8,1,0,0,13,0.9,186,...,2,1152,1632,1933,8,1,19,0,1,1
997,998,1185,0,1.4,0,1,1,8,0.5,80,...,12,477,825,1223,5,0,14,1,0,0
998,999,1533,1,0.5,1,0,0,50,0.4,171,...,12,38,832,2509,15,11,6,0,1,0


In [4]:
def feature_engineering(df, fit_scaler=True, scaler=None, bin_edges=None):
    # Calculate pixel density
    
    df = df[['battery_power', 'px_height', 'px_width', 'ram']]
    df = df.fillna(df.median())
    df['pixel_density'] = df['px_height'] + df['px_width']
    
    # Log transformation
    df['log_ram'] = np.log1p(df['ram'])
    df['log_battery_power'] = np.log1p(df['battery_power'])

    # Binning logic
    if bin_edges is None:  # During training, calculate the bin edges
        df['battery_power_bins'], battery_power_bins_edges = pd.qcut(df['battery_power'], q=4, labels=False, retbins=True, duplicates='drop')
        df['ram_bins'], ram_bins_edges = pd.qcut(df['ram'], q=4, labels=False, retbins=True, duplicates='drop')
        
        # Save these edges for later use
        bin_edges = {
            'battery_power': battery_power_bins_edges,
            'ram': ram_bins_edges
        }
    else:  # During prediction, reuse the saved bin edges
        df['battery_power_bins'] = pd.cut(df['battery_power'], bins=bin_edges['battery_power'], labels=False, include_lowest=True)
        df['ram_bins'] = pd.cut(df['ram'], bins=bin_edges['ram'], labels=False, include_lowest=True)

    # Drop original columns
    df = df.drop(columns=['px_height', 'px_width'], axis=1)
    df = df.fillna(df.median())
    # List of numerical features to scale
    numerical_features = ['battery_power', 'pixel_density', 'log_ram', 'log_battery_power', 'ram']  # Adjust as needed

    # Scale numerical features
    if fit_scaler:
        scaler = StandardScaler()
        df[numerical_features] = scaler.fit_transform(df[numerical_features])
    else:
        df[numerical_features] = scaler.transform(df[numerical_features])

    return df, scaler, bin_edges  # Return bin_edges and scaler when training




In [5]:
import joblib
scaler = joblib.load('scaler.pkl')
bin_edges=joblib.load('bin_edges.pkl')

In [6]:
  # Filling NaNs in numeric columns with median

X_test ,_, _ = feature_engineering(test.head(1), fit_scaler=False, scaler=scaler, bin_edges=bin_edges)

In [7]:
X_test.isna().sum()

battery_power         0
ram                   0
pixel_density         0
log_ram               0
log_battery_power     0
battery_power_bins    0
ram_bins              0
dtype: int64

In [8]:
X_test

,battery_power,ram,pixel_density,log_ram,log_battery_power,battery_power_bins,ram_bins
0,-0.439893,1.234474,-0.338031,0.996695,-0.255425,1,3


In [9]:
import joblib
svc_model = joblib.load('svc_model.pkl')

In [10]:
pred=X_test.head(1)

# Make predictions
predictions = svc_model.predict(pred)

# Print the predictions
print("Predictions for the first 10 rows:")
print(predictions)

Predictions for the first 10 rows:
[3]
